# Reappraisal Training For Linguistic Distancing and Emotion Regulation


## Setup
```bash
> pipenv shell  #Generates a new virtual environment based on Pipfile
> pipenv install # Installs the packages in Pipfile.lock (Use --dev) to also install dev packages
```
## Included Datasets
- LDHII 
- 
- Emobank
**Sources**
-  [Sentiment Analysis Text Classification Tutorial](https://www.youtube.com/watch?v=8N-nM3QW7O0)
- [Using Catalyst for Training Organization](https://github.com/catalyst-team/catalyst)



In [1]:
# TODO: Add Open in Colab Button
# TODO: Write scripts for running as CLI in pipfile
# TODO: hyperparameter search

In [2]:
import os
import numpy as np
import pandas as pd
import torch
# !pipenv install pandas

In [3]:
from datasets import ReadInstruction
from transformers import DistilBertTokenizer

# Enable GPU usage, if we can.
if torch.cuda.is_available():
    print("Enabling GPU usage")
    device = torch.device("cuda:0")
    IS_GPU = True
else:
    print("No GPU available, running on CPU")
    device = torch.device("cpu") # Note: macOS incompatible with NVIDIA GPUs
    IS_GPU = False
    
# Constants and environment setup
# TODO: Set up env files for dev and "prod"
#Casing can matter for sentiment analysis ("bad" vs. "BAD")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")

tokenize_func = lambda batch: tokenizer(
    batch['response'],
    add_special_tokens=True,
    padding="max_length",
    truncation=True)

PRETRAINED_MODEL_NAME = 'distilbert-base-cased'

No GPU available, running on CPU


### LDH Dataset Imports

In [4]:
from src.LDHData import LDHData

data = LDHData()

### IMDB Dataset Import

### EmoBank Dataset Import

In [5]:
from torch import nn, optim
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import Features, Sequence, Value, DatasetDict
# Tokenize the datasets.
# Add score to the model inputs so we can calculate loss.

# encoded_ds = data.encode(tokenizer, train_ds)

# encoded_ds['train'].features
# data.set_tokenizer(tokenizer)
# train_ds = data.get_train_far_data()

encoded_ds = data.encode_datasets(tokenizer, batched=True, batch_size=16)
encoded_ds

{'train': DatasetDict({
     far: Dataset({
         features: ['attention_mask', 'input_ids', 'response', 'score'],
         num_rows: 13470
     })
     obj: Dataset({
         features: ['attention_mask', 'input_ids', 'response', 'score'],
         num_rows: 13470
     })
 }),
 'eval': DatasetDict({
     far: Dataset({
         features: ['__index_level_0__', 'addcode', 'attention_mask', 'input_ids', 'level_0', 'response'],
         num_rows: 22972
     })
     obj: Dataset({
         features: ['__index_level_0__', 'addcode', 'attention_mask', 'input_ids', 'level_0', 'response'],
         num_rows: 22972
     })
 })}

## Generate NN Model

In [14]:
from transformers import TrainingArguments, Trainer, DistilBertModel
from torch.utils.data import DataLoader

from src.ReappModel import ReappModel

# Create the training model.
# TODO: Suppress initialization errors.
model = ReappModel(DistilBertModel, PRETRAINED_MODEL_NAME)

num_train_epochs = 3 if IS_GPU else 1

# Define the parameters under which the model will be trained.
# By default, uses an AdamW optimizer w/ linear warmup.

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)
encoded_train = encoded_ds['train']['far']
encoded_eval  = encoded_ds['eval']['far']

print(len(encoded_eval))

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,                  
    train_dataset=encoded_train,      
    eval_dataset=encoded_eval         
)

22972


In [16]:
encoded_train_step = encoded_train.select(range(16))
input_ids = encoded_train_step['input_ids']
attention_mask = encoded_train_step['attention_mask']
score = encoded_train_step['score'],

trainer.training_step(model, {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "score": score
    }
)



torch.Size([16]) torch.Size([16])


TypeError: prediction_step() missing 1 required positional argument: 'prediction_loss_only'

In [8]:
# Model Evaluation: Parse the TrainOutput Object 

In [18]:
trainer.prediction_step(model, {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "score": score
    }
, False)
# TODO: fix the result of the forward method so it properly returns the result of a prediction step


torch.Size([16]) torch.Size([16])


(None,
 (tensor(1.2515e+09),
  tensor([[[0.2974, 0.0321, 0.0000,  ..., 0.0000, 0.1631, 0.0018],
           [0.0479, 0.0141, 0.3758,  ..., 0.0000, 0.3087, 0.0000],
           [0.1139, 0.0000, 0.1657,  ..., 0.0000, 0.2711, 0.0999],
           ...,
           [0.1065, 0.0070, 0.0000,  ..., 0.0000, 0.3119, 0.0000],
           [0.2112, 0.0000, 0.0000,  ..., 0.0898, 0.3542, 0.0000],
           [0.1867, 0.0075, 0.0000,  ..., 0.0000, 0.2876, 0.0000]],
  
          [[0.2513, 0.0061, 0.0000,  ..., 0.0000, 0.2053, 0.2262],
           [0.1464, 0.0000, 0.0622,  ..., 0.0000, 0.5484, 0.0700],
           [0.0000, 0.0000, 0.3078,  ..., 0.0834, 0.0246, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.1892, 0.3698, 0.0000],
           [0.0000, 0.0129, 0.0000,  ..., 0.0833, 0.2472, 0.0000],
           [0.0154, 0.0000, 0.0000,  ..., 0.0366, 0.3223, 0.0000]],
  
          [[0.0723, 0.0000, 0.0222,  ..., 0.0000, 0.2573, 0.1352],
           [0.0000, 0.0000, 0.1659,  ..., 0.1130, 0.4028, 0.